### 期末作业

**Notice!!!!!**

1. 请将期末作业于**2021年1月7日，晚上24:00**之前提交至邮箱 liyan_zjgsu@163.com
2. 请以jupyter notebook的形式提交, 文件的命名方式为**PythonFinal_班级_学号__姓名**（原始文件或压缩文件均需以此方式命名）
3. 特别说明：为了便于给分以及避免漏判，请大家在写完代码之后注意：  
（1）将代码按照各小题**模块化**  
（2）尽可能多的写上**注释**，特别是对每个小题具体解法以及生成结果的说明


**格式举例**

```
#### Answer for Q1 

import tushare as ts
……

## Q1.1 请从tushareAPI获取2017年初至今所有银行股的收盘价。  

your code block ## 行间注释

print(result) ## 输出结果

## Q1.2 ……

……

#### Answer for Q2
……
## Q2.1
……
## Q2.2
……
```


**Q1.**

    1. 生成一个元素取值在[0, 100]中随机分布的10x4的二维数组A，生成一个元素取值符合正态分布的一维数组B，数组长度为40
    2. 将B重塑为4x10的二维数组，计算矩阵C = AxB，C的逆矩阵以及C的行列式。
    3. (a)以列表的形式返回矩阵C每一行中大于该行均值的数字 (b)以二维数组的形式返回上述数字所处的位置。
    4. 从二维数组C生成一个DataFrame，将列名重命名为字母A~J
    5. 将该DataFrame保存到本地C盘根目录下，文件名为rand_num.csv
    
**解答：**

In [ ]:
#### Answer for Q1
## Q1.1
import numpy as np
from random import randint
import pandas as pd

A = np.zeros([10, 4])
for x in range(A.shape[0]):
    for y in range(A.shape[1]):
        A[x, y] = randint(0, 100)
print(A)

B = np.random.normal(size=[40])
print(B)

## Q1.2
B = np.resize(B, new_shape=[4, 10])
C = np.matmul(A, B)
print(C)

C_I = np.linalg.inv(C)
print(C_I)

C_det = np.linalg.det(C)
print(C_det)

## Q1.3
# (a)
C_mean = np.mean(C)
a_list = []
for x in range(C.shape[0]):
    for y in range(C.shape[1]):
        if C[x, y] > C_mean:
            a_list.append(C[x, y])
print(a_list)
# (b)
b_list = []
for x in range(C.shape[0]):
    for y in range(C.shape[1]):
        if C[x, y] > C_mean:
            b_list.append((x, y))
print(b_list)

## Q1.4
df = pd.DataFrame(C)
df.columns = list('ABCDEFGHIJ')
print(df)

## Q1.5
df.to_csv("C:\\rand_num.csv")


**Q2.**

    1. 从tushareAPI提取所有上市公司的股票列表，从上市日期在2010年之前的股票中随机选出10支，从tushare获得它们的日线行情数据
    2. 将这10支股票的成交量按照trade_date连接，取数据并集的方式合并成一个dataframe，列名为股票代码，行索引为trade_date
    3. 随机选出两支股票，(a)对它们的收益率序列进行独立两样本t检验 (b)对它们的收盘价序列进行线性回归并输出结果
    4. 对2中的表格，计算每一行的均值，并取出均值最大的10行
    5. 将2中表格的nan值填充为对应列的均值，可视化该表格为折线图

**解答：**

In [ ]:
#### Answer for Q2
## Q2.1
import numpy as np
import pandas as pd
import tushare as ts
import random
from scipy.stats import ttest_ind, levene

# 请在 tushare.pro 网站注册并且告知学生身份，可以取得你的token

tushare_token = '1c8b06446534ae510c8c68e38fc248b99f89ac3814cb55645ae2be72'
pro = ts.pro_api(tushare_token)

data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')

data = data[data['list_date'] > '20100101']  # 过滤掉上市时间不在2010年之前的股票

ts_code_list = data.sample(n=10)['ts_code'].values.tolist()  # 随机选取10只股票

daily_list = []
for ts_code in ts_code_list:
    daily_list.append(pro.daily(ts_code=ts_code))

## Q2.2

#  获取vol序列,并以日期为索引,以股票名命名
vol_list = []
for daily in daily_list:
    daily.set_index(['trade_date'], inplace=True)
    vol = daily['vol']
    vol.name = daily['ts_code'][0]
    vol_list.append(vol)

vol_df = vol_list[0]
for i in range(1, len(vol_list)):
    vol_df = pd.merge(vol_df, vol_list[i], on="trade_date", how="inner")
print(vol_df)

## Q2.3
# (a)

# pct_change序列,同上
pct_change_list = []
for daily in daily_list:
    pct_change = daily['pct_chg']
    pct_change.name = daily['ts_code'][0]
    pct_change_list.append(pct_change)

pct_change_df = pct_change_list[0]
for i in range(1, len(pct_change_list)):
    pct_change_df = pd.merge(pct_change_df, pct_change_list[i], on="trade_date", how="inner")
print(pct_change_df)

two_ = random.sample(list(pct_change_df.columns), k=2)
pct_change_df_two = pct_change_df[two_]

# 计算两只股票的t检验
X = pct_change_df_two[two_[0]]
Y = pct_change_df_two[two_[1]]

l_xy = levene(X, Y)

if l_xy.pvalue > 0.05:
    t_xy = ttest_ind(X, Y, equal_var=True)
else:
    t_xy = ttest_ind(X, Y, equal_var=False)
print("T_statistic:", t_xy.statistic, "T_pvalue:", t_xy.pvalue)

# (b)

# 用线性回归预测下一天的股价
close_list = []
for daily in daily_list:
    series = daily['close']
    series.name = daily['ts_code'][0]
    close_list.append(series)

close_df = close_list[0]
for i in range(1, len(close_list)):
    close_df = pd.merge(close_df, close_list[i], on="trade_date", how="inner")
print(close_df)

close_df_two = close_df[two_]
X = close_df_two[two_[0]]
Y = close_df_two[two_[1]]

import statsmodels.api as sm
from statsmodels import regression


def linear_predict(series):
    def regress_y(y):
        x = np.arange(0, len(y))
        x = sm.add_constant(x)
        model = regression.linear_model.OLS(y, x).fit()
        return model

    model = regress_y(series)
    b = model.params[0]
    k = model.params[1]
    return k * (len(series) + 1) + b


X_predict = linear_predict(X)
Y_predict = linear_predict(Y)
print("X_predict:", X_predict)
print("Y_predict:", Y_predict)

## Q2.4

# 选取top10
vol_df24 = vol_df.copy()
vol_df24['mean'] = vol_df24.mean(axis=1)
vol_df24_mean_top10_index = vol_df24.loc[vol_df24['mean'].nlargest(n=10).index]
print(vol_df24_mean_top10_index)

## Q2.5
vol_df.apply(lambda x: x.fillna(x.mean()), axis=0)
vol_df.plot()

**Q3.**

    1. 从 url=http://stock.finance.sina.com.cn/stock/go.php/vReport_List/kind/lastest/index.phtml?p=1 爬取1-50页的标题、报告类型、发布日期、机构四项信息，并将它们存入DataFrame中

|title|type|date|institution|
| -- | -- | -- | -- |
| 标题1 | …… | …… | …… |
| 标题2 | …… | …… | …… |
……

    2. 将该DataFrame存入本地的MySQL数据库中，存入的数据库名为sina，表名为report
    3. 利用SQL语句统计上表中不同类型的记录数量，并从数据库中筛选 报告类型 为 行业 的表格
    4. (a)对1中表格的 标题 这一列进行情感评分，并存入原始表格的sentiment列 (b)将情感评分的值按照 报告类型 求平均
    5. 点击1中url页面里的标题，可以看到每篇研报的具体文字内容，请利用tf-idf和textrank算法提取该url页面所有研报内容权重最大的关键词，并以DataFrame格式输出
    6. 绘制上述url页面中第一篇研报的词云图，注意过滤停用词。

|title|tf-idf|textrank|
| -- | -- | -- |
| 标题1 | 关键字 | 关键字 |
| 标题2 | 关键字 | 关键字 |
……

**解答：**

In [ ]:
#### Answer for Q3
## Q3.1
url = "http://stock.finance.sina.com.cn/stock/go.php/vReport_List/kind/lastest/index.phtml?p=1"
url2 = "http://stock.finance.sina.com.cn/stock/go.php/vReport_List/kind/lastest/index.phtml?p=2"
import pandas as pd
import requests
from bs4 import BeautifulSoup


def get_list(url):
    r = requests.get(url)
    # 用BeautifulSoup解析网页,用GB18030编码,不然会乱吗
    content = BeautifulSoup(r.content.decode("gb18030"), "html.parser")
    main_text = content.find("div", class_="main")
    tr_list = main_text.find_all("tr")[2:-1]
    return tr_list


tr_list = get_list(url) + get_list(url2)[:10]


def get_attr(tr):
    title = tr.find("a")['title']
    type = tr.contents[5].contents[0]
    date = tr.contents[7].contents[0]
    insitution = tr.find("span").contents[0]
    return title, type, date, insitution


data_list = []
for tr in tr_list:
    data_list.append(get_attr(tr))
df = pd.DataFrame(data_list, columns=['title', 'type', 'date', 'insitution'])
print(df)

## Q3.2

import pymysql

db = pymysql.connect(host="localhost", user="root", password="123456", database="sina")
cursor = db.cursor()
for data in data_list:
    sql = "insert into report(title, type, date, insitution) values ('%s', '%s', '%s', '%s')" % (
        data[0], data[1], data[2], data[3])
    cursor.execute(sql)

db.commit()

## Q3.3

sql = "select type, count(*) from report group by type"
res = cursor.execute(sql)
print(res)

sql = "select * from report where type = '报告'"
res = cursor.execute(sql)
print(res)

## Q3.4
# (a）
from snownlp import SnowNLP

# sentiment 打分
df['sentiment'] = df['title'].apply(lambda x: SnowNLP(x).sentiments)

# (b)
sentiment_mean = df['sentiment'].mean(axis=0)
print(sentiment_mean)


## Q3.5
def get_content(tr):
    url = "http:" + tr.find("a")['href']
    r = requests.get(url)
    content = BeautifulSoup(r.content.decode("gb18030"), "html.parser")
    main_text = content.find("div", class_="blk_container")
    text = main_text.find("p").text
    return text.strip().replace("\n", "").replace(" ", "").replace("\t", "").replace("\r", "").replace(" ", "")


text_list = []
text_all = ""

for tr in tr_list:
    text = get_content(tr)
    text_list.append(text)
    text_all += text

import jieba.analyse as analyse

print('TF-IDF')
tf_idf_keyword_list = []
for text in text_list:
    for keyword, weight in analyse.extract_tags(text, topK=1, withWeight=True):
        tf_idf_keyword_list.append(keyword)
print('TextRank')
text_rank_keyword_list = []
for text in text_list:
    for keyword, weight in analyse.textrank(text_all, topK=1, withWeight=True):
        text_rank_keyword_list.append(keyword)

tf_idf_keyword_series = pd.Series(tf_idf_keyword_list)
text_rank_keyword_series = pd.Series(text_rank_keyword_list)
print(tf_idf_keyword_series)
print(text_rank_keyword_series)

## Q3.6
from wordcloud import WordCloud
import matplotlib.pyplot as plt

my_wordcloud = WordCloud(font_path="SimHei.ttf").generate(text_list[0])  # 生成词云

plt.imshow(my_wordcloud)  # 绘制词云图片
plt.axis("off")  # 不显示坐标轴
plt.show()  # 显示图片

**Q4.** 

    1. 请从tushareAPI获取2017年初至今所有银行股的收盘价。  
    2. 计算上述股票收盘价的收益率序列的相关矩阵，并绘制热图。  
    3. 请在最大回撤最小的前50%支股票中找出下行风险最小的前50%支股票。  
    4. 绘制总标题为中文“银行股”，2x1的子图，调整图片大小为20x20。
    子图一：将4中筛选出的股票价格序列起始点均平移到0点后，以日期为横坐标，收盘价为纵坐标绘制折线图，不显示横坐标刻度和标签，在画面左上角添加图例，为图片添加网格并设置50%的透明度。  
    子图二：计算个股收盘价序列的均值作为行业指数，计算该行业指数的6日的指数平均线ema，以及6日移动窗口中的标准差std6，在图中绘制原始行业指数和ema，并以ema+std6和ema-std6为上下边界进行填充，填充的颜色为红色，透明度为0.1，请在图中以适当的间距显示日期。  
    
**解答：**

In [ ]:
#### Answer for Q4
## Q4.1
import math

import numpy as np
import pandas as pd
import tushare as ts

tushare_token = '1c8b06446534ae510c8c68e38fc248b99f89ac3814cb55645ae2be72'
pro = ts.pro_api(tushare_token)
data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
data = data[data['list_date'] > '20170101']
data = data[data['industry'] == '银行']
ts_code_list = data['ts_code'].values.tolist()

daily_list = []
for ts_code in ts_code_list:
    daily_list.append(pro.daily(ts_code=ts_code).dropna())

close_list = []
for daily in daily_list:
    daily = daily.set_index(['trade_date'], inplace=False)
    close_list.append(daily['close'].rename(daily['ts_code'][0]))
close_df = pd.concat(close_list, axis=1)
print(close_df)

## Q4.2
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

for daily in daily_list:
    close_pct_change = daily[['pct_chg', 'close']]
    f, ax = plt.subplots(figsize=(14, 10))
    corrdf = close_pct_change.corr()
    sns.heatmap(corrdf, cmap='RdBu', linewidths=0.05, ax=ax)

    # 设置Axes的标题
    ax.set_title(daily['ts_code'][0])
    plt.show()

pct_change_list = []
for daily in daily_list:
    pct_change = daily['pct_chg']
    pct_change.name = daily['ts_code'][0]
    pct_change_list.append(pct_change)

pct_change_df = pd.concat(pct_change_list, axis=1)

f, ax = plt.subplots(figsize=(14, 10))
corrdf = pct_change_df.corr()
sns.heatmap(corrdf, cmap='RdBu', linewidths=0.05, ax=ax)

# 设置Axes的标题
ax.set_title('Correlation between stocks')
plt.show()


## Q4.3
def MaxDrawdown(returns):
    l = np.argmax((np.maximum.accumulate(returns) - returns) / np.maximum.accumulate(returns))
    k = np.argmax(returns[:l])
    return (returns[k] - returns[l]) / (returns[l])


def cal_half_def(returns):
    for i in range(len(returns)):
        returns[i] = returns[0] - returns[i]
    mu = returns.mean()  # 这里使用的是均值
    temp = returns[returns < mu]
    half_deviation = (sum((temp - mu) ** 2) / len(temp)) ** 0.5
    return half_deviation


stocks = []
for daily in daily_list:
    close_list = daily['close'].values.tolist()
    close_list.reverse()
    maximum_pullback = MaxDrawdown(close_list)
    half_deviation = cal_half_def(np.array(close_list))
    print(maximum_pullback, half_deviation)
    stocks.append((daily['ts_code'][0], maximum_pullback, half_deviation))
stocks.sort(key=lambda x: x[1])
stocks = stocks[:len(stocks) // 2]
stocks.sort(key=lambda x: x[2])
stocks = stocks[:len(stocks) // 2]
for stock in stocks:
    print(stock[0])

## Q4.4
from matplotlib import pyplot as plt

# 绘制子图
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(20, 20))
close_df = close_df.dropna().iloc[::-1]
close_df_sub = close_df - close_df.iloc[0, :]

ax[0].plot(close_df_sub)
ax[0].set_title('Close Price')
close_df['mean'] = close_df.mean(axis=1)


def get_ema_std(six_days):
    ema = sum(six_days) / len(six_days)
    std = sum((six_days - ema) ** 2) / len(six_days)
    return ema, std


ema_list = []

for i in range(6, len(close_df['mean'])):
    six_days = close_df['mean'][i - 6:i]
    ema, std = get_ema_std(six_days)
    ema_list.append(ema)

# 一些精细的设置 学习成本太高
close_df = close_df[6:]
close_df['ema'] = ema_list
close_df_mean_ema = close_df[['mean', 'ema']]
close_df_mean_ema.plot(ax=ax[1])
plt.show()

**Q++**

    画出000001.SZ的k线图，对其使用MACD策略，计算净值曲线，夏普比率，最大回撤及下行风险。
    
**解答：**

In [ ]:
#### Answer for Q++
import math

import matplotlib.pyplot as plt
import numpy as np
import talib
import tushare as ts
from matplotlib import ticker
from mplfinance.original_flavor import candlestick_ochl

tushare_token = '1c8b06446534ae510c8c68e38fc248b99f89ac3814cb55645ae2be72'
pro = ts.pro_api(tushare_token)
code = "000001.SZ"
df = pro.daily(ts_code=code)
df.drop(df.columns[0], axis=1, inplace=True)

df2 = df.query('trade_date >= "20171001"').reset_index()  # 选取2017年10月1日后的数据
df2 = df2.sort_values(by='trade_date', ascending=True)  # 原始数据按照日期降序排列
df2['dates'] = np.arange(0, len(df2))  # len(df2)指记录数
fig, ax = plt.subplots(figsize=(20, 9))
fig.subplots_adjust(bottom=0.2)  # 控制子图
###candlestick_ochl()函数的参数
# ax 绘图Axes的实例
# quotes  序列（时间，开盘价，收盘价，最高价，最低价） 时间是float类型，date必须转换为float
# width    图像中红绿矩形的宽度,代表天数
# colorup  收盘价格大于开盘价格时的颜色
# colordown   低于开盘价格时矩形的颜色
# alpha      矩形的颜色的透明度
candlestick_ochl(ax, quotes=df2[['dates', 'open', 'close', 'high', 'low']].values,
                 width=0.55, colorup='r', colordown='g', alpha=0.95)
date_tickers = df2['trade_date'].values


def format_date(x, pos):
    if (x < 0) or (x > len(date_tickers) - 1):
        return ''
    return date_tickers[int(x)]


ax.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))  # 按一定规则选取并在水平轴上显示时间刻度
plt.xticks(rotation=30)  # 设置日期刻度旋转的角度
ax.set_ylabel('交易价格')
plt.title(code)
plt.grid(True)  # 添加网格，可有可无，只是让图像好看一些
plt.xlabel('交易日期')
plt.show()

## MACD
close = df2['close'].values
macd = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
print(macd)

##


## 计算夏普比率
df['ex_pct_close'] = df['pct_chg'] - 0.04 / 252
sharpeRatio = (df['ex_pct_close'].mean() * math.sqrt(252)) / df['ex_pct_close'].std()
print("sharpeRatio:", sharpeRatio)


## 最大回撤
def MaxDrawdown(returns):
    l = np.argmax((np.maximum.accumulate(returns) - returns) / np.maximum.accumulate(returns))
    k = np.argmax(returns[:l])
    return (returns[k] - returns[l]) / (returns[l])


maximumPullback = MaxDrawdown(close)
print("maximumPullback:", maximumPullback)


def cal_half_def(returns):
    for i in range(len(returns)):
        returns[i] = returns[0] - returns[i]
    mu = returns.mean()  # 这里使用的是均值
    temp = returns[returns < mu]
    half_deviation = (sum((temp - mu) ** 2) / len(temp)) ** 0.5
    return half_deviation

# 下行风险
half_deviation = cal_half_def(np.array(close))
print("half_deviation:", half_deviation)
